In [87]:
import pandas as pd
import numpy as np

In [88]:
df = pd.read_csv('./dataset/astana_apartments_ready.csv')
df

,rooms,area,living_area,kitchen_area,floor,total_floors,price_usd,price_per_m2,house_type,ceiling_height,...,security_high,bathroom_count,bathroom_type,balcony_type,wooden_floor,floor_relative,living_ratio,kitchen_ratio,building_age,building_stage
0,3,89.30,63.670196,11.800000,4.0,5.0,88424,990,brick,2.8,...,0,2,multiple,multiple,0,0.800000,0.712992,0.132139,12,0
1,2,72.10,51.406731,14.729211,-1.0,-1.0,56984,790,monolithic,2.7,...,1,2,multiple,loggia,0,1.000000,0.712992,0.204289,28,0
2,4,106.77,76.126168,11.130000,8.0,9.0,149338,1398,monolithic,3.0,...,0,2,unknown,unknown,0,0.888889,0.712992,0.104243,3,0
3,2,39.00,27.806692,7.967257,2.0,2.0,27509,705,brick,2.7,...,1,1,combined,loggia,0,1.000000,0.712992,0.204289,5,0
4,2,39.00,27.806692,7.967257,1.0,8.0,45587,1168,brick,2.7,...,0,1,unknown,unknown,0,0.125000,0.712992,0.204289,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15551,5,131.00,93.401967,26.761812,2.0,9.0,204358,1559,brick,3.0,...,0,2,unknown,unknown,0,0.222222,0.712992,0.204289,4,0
15552,2,65.40,46.629684,10.600000,7.0,12.0,92354,1412,monolithic,3.0,...,1,2,multiple,multiple,0,0.583333,0.712992,0.162080,1,0
15553,2,39.00,27.806692,7.967257,2.0,12.0,60914,1561,brick,2.7,...,0,1,combined,balcony,0,0.166667,0.712992,0.204289,2,0
15554,2,69.80,49.766849,9.200000,2.0,9.0,72704,1041,brick,2.5,...,0,1,separate,loggia,0,0.222222,0.712992,0.131805,6,0


In [89]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
def evaluate_model(model, model_name, X_train, y_train, X_val, y_val, X_test, y_test):
    print(f"\n{'='*60}")
    print(f"Results for: {model_name}")
    print(f"{'='*60}")

    y_pred_train = np.expm1(model.predict(X_train))
    y_train_orig = np.expm1(y_train)

    y_pred_val = np.expm1(model.predict(X_val))
    y_val_orig = np.expm1(y_val)

    y_pred_test = np.expm1(model.predict(X_test))
    y_test_orig = np.expm1(y_test)

    results = {}

    for name, y_true, y_pred in [
        ('Train', y_train_orig, y_pred_train),
        ('Validation', y_val_orig, y_pred_val),
        ('Test', y_test_orig, y_pred_test)
    ]:
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        results[name] = {'rmse': rmse, 'mae': mae, 'r2': r2, 'mape': mape}

        print(f"\n{name} Set:")
        print(f"  RMSE: ${rmse:,.2f}")
        print(f"  MAE:  ${mae:,.2f}")
        print(f"  R²:   {r2:.4f}")
        print(f"  MAPE: {mape:.2f}%")

        bins = [0, 50000, 80000, 110000, np.inf]
        labels = ['<50k', '50–80k', '80–110k', '>110k']

        df_err = pd.DataFrame({
            'price': y_true,
            'pred': y_pred
        })

        df_err['bin'] = pd.cut(df_err['price'], bins=bins, labels=labels)

        df_err['abs_err'] = np.abs(df_err['price'] - df_err['pred'])

        bin_summary = (
        df_err
        .groupby('bin', observed=True)
        .agg(
            mean_abs_error=('abs_err', 'mean'),
            mean_abs_pct_error=('abs_err',  lambda x: np.mean(x / df_err.loc[x.index, 'price']) * 100)
            )
        )

        print("\nMean Error by Price Bin")
        print("-" * 30)
        print(bin_summary)

    return results

Features

In [90]:
target = 'price_usd'

numeric_features = [
    'rooms', 'area', 'living_area', 'kitchen_area', 'floor', 'total_floors',
    'building_age', 'ceiling_height', 'latitude', 'longitude',
    'has_window_grills', 'security_high', 'bathroom_count', 'wooden_floor',
    'floor_relative', 'living_ratio', 'kitchen_ratio'
]

categorical_features = [
    'house_type', 'parking', 'furniture', 'district', 'bathroom_type', 'balcony_type',
'condition'
]

One-Hot Encoding

In [91]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_encoded = ohe.fit_transform(df[categorical_features])
cat_encoded_df = pd.DataFrame(cat_encoded, columns=ohe.get_feature_names_out(categorical_features))

Final dataset and log of target

In [92]:
X = pd.concat([df[numeric_features].reset_index(drop=True), cat_encoded_df.reset_index(drop=True)], axis=1)

y = np.log1p(df[target])

Split - Train / Val / Test

In [93]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Size train: {X_train.shape[0]} rows")
print(f"Size validation: {X_val.shape[0]} rows")
print(f"Size test: {X_test.shape[0]} rows")

Size train: 10889 rows
Size validation: 2333 rows
Size test: 2334 rows


In [94]:
import xgboost as xgb

print("Training XGBoost Model...")
print("-"*30)
xgb_model = xgb.XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method='hist'
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=50
)

xgb_metrics = evaluate_model(xgb_model, 'XGBRegressor', X_train, y_train, X_val, y_val, X_test, y_test)

Training XGBoost Model...
------------------------------
[0]	validation_0-rmse:0.51043
[50]	validation_0-rmse:0.17398
[100]	validation_0-rmse:0.15060
[150]	validation_0-rmse:0.14383
[200]	validation_0-rmse:0.14041
[250]	validation_0-rmse:0.13836
[300]	validation_0-rmse:0.13674
[350]	validation_0-rmse:0.13576
[400]	validation_0-rmse:0.13467
[450]	validation_0-rmse:0.13409
[500]	validation_0-rmse:0.13359
[550]	validation_0-rmse:0.13304
[600]	validation_0-rmse:0.13259
[650]	validation_0-rmse:0.13225
[700]	validation_0-rmse:0.13205
[750]	validation_0-rmse:0.13181
[800]	validation_0-rmse:0.13173
[850]	validation_0-rmse:0.13153
[900]	validation_0-rmse:0.13134
[950]	validation_0-rmse:0.13115
[999]	validation_0-rmse:0.13101

Results for: XGBRegressor

Train Set:
  RMSE: $5,762.38
  MAE:  $3,606.10
  R²:   0.9873
  MAPE: 4.15%

Mean Error by Price Bin
------------------------------
         mean_abs_error  mean_abs_pct_error
bin                                        
<50k        1486.569275   

In [95]:
import lightgbm as lgb

print("Training LightGBM Model...")
print("-"*30)

lgb_model = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6,
    num_leaves=50,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)

lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=50)]
)

lgb_metrics = evaluate_model(lgb_model, 'LightGBM', X_train, y_train, X_val, y_val, X_test, y_test)

Training LightGBM Model...
------------------------------
Training until validation scores don't improve for 50 rounds
[50]	valid_0's l2: 0.0306458
[100]	valid_0's l2: 0.0225527
[150]	valid_0's l2: 0.0206977
[200]	valid_0's l2: 0.0198583
[250]	valid_0's l2: 0.0193784
[300]	valid_0's l2: 0.0189886
[350]	valid_0's l2: 0.018672
[400]	valid_0's l2: 0.0184531
[450]	valid_0's l2: 0.0182687
[500]	valid_0's l2: 0.0181232
[550]	valid_0's l2: 0.0180291
[600]	valid_0's l2: 0.0179133
[650]	valid_0's l2: 0.0178327
[700]	valid_0's l2: 0.0177812
[750]	valid_0's l2: 0.0177199
[800]	valid_0's l2: 0.017657
[850]	valid_0's l2: 0.0175589
[900]	valid_0's l2: 0.0175297
[950]	valid_0's l2: 0.0175315
Early stopping, best iteration is:
[920]	valid_0's l2: 0.0175081

Results for: LightGBM

Train Set:
  RMSE: $8,636.86
  MAE:  $5,260.63
  R²:   0.9714
  MAPE: 5.99%

Mean Error by Price Bin
------------------------------
         mean_abs_error  mean_abs_pct_error
bin                                        
<50k 

In [96]:
from catboost import CatBoostRegressor

print("Training CatBoost Model...")
print("-"*30)

# For CatBoost, it's better to use original categorical features
# Create a dataset with categorical features as strings
X_cat = pd.concat([
    df[numeric_features].reset_index(drop=True),
    df[categorical_features].reset_index(drop=True).astype(str)
], axis=1)

#Split new dataset
X_train_cat, X_temp_cat, y_train_cat, y_temp_cat = train_test_split(
    X_cat, y, test_size=0.3, random_state=42
)
X_val_cat, X_test_cat, y_val_cat, y_test_cat = train_test_split(
    X_temp_cat, y_temp_cat, test_size=0.5, random_state=42
)

cat_features_indices = [X_cat.columns.get_loc(col) for col in categorical_features]

print("Checking categorical features for float dtypes:")
for idx in cat_features_indices:
    col_name = X_train_cat.columns[idx]
    dtype = X_train_cat[col_name].dtype
    dval = X_train_cat[col_name]

    if dtype in ['float64', 'float32', 'float16']:
        print(f"⚠️  Column {idx} ({col_name}): {dtype} <- PROBLEM!")
        print(f"   Sample values: {X_train_cat[col_name].head()}")
        print(f"   Has NaN: {X_train_cat[col_name].isna().any()}")
        print()
    else:
        print(f"✓  Column {idx} ({col_name}): {dtype}")

catboost_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3,
    subsample=0.8,
    random_seed=42,
    cat_features=cat_features_indices,
    verbose=50,
    early_stopping_rounds=50
)

catboost_model.fit(
    X_train_cat, y_train_cat,
    eval_set=(X_val_cat, y_val_cat),
    use_best_model=True
)

cat_metrics = evaluate_model(catboost_model, 'CatBoost', X_train_cat, y_train_cat, X_val_cat, y_val_cat, X_test_cat, y_test_cat)

Training CatBoost Model...
------------------------------
Checking categorical features for float dtypes:
✓  Column 17 (house_type): object
✓  Column 18 (parking): object
✓  Column 19 (furniture): object
✓  Column 20 (district): object
✓  Column 21 (bathroom_type): object
✓  Column 22 (balcony_type): object
✓  Column 23 (condition): object
0:	learn: 0.5118852	test: 0.5125279	best: 0.5125279 (0)	total: 20.6ms	remaining: 20.6s
50:	learn: 0.1845140	test: 0.1910360	best: 0.1910360 (50)	total: 334ms	remaining: 6.21s
100:	learn: 0.1558530	test: 0.1646263	best: 0.1646263 (100)	total: 610ms	remaining: 5.43s
150:	learn: 0.1446659	test: 0.1559606	best: 0.1559606 (150)	total: 881ms	remaining: 4.96s
200:	learn: 0.1357551	test: 0.1495597	best: 0.1495597 (200)	total: 1.15s	remaining: 4.59s
250:	learn: 0.1290943	test: 0.1456210	best: 0.1456210 (250)	total: 1.49s	remaining: 4.45s
300:	learn: 0.1237262	test: 0.1427499	best: 0.1427499 (300)	total: 1.9s	remaining: 4.41s
350:	learn: 0.1190226	test: 0.1407

In [97]:
print("\n" + "-"*30)
print("Basic models comparison")
print("-"*30)

results = pd.DataFrame({
    'Model': ['XGBoost', 'LightGBM', 'CatBoost'],
    'RMSE': [xgb_metrics['Test']['rmse'], lgb_metrics['Test']['rmse'], cat_metrics['Test']['rmse']],
    'R²': [xgb_metrics['Test']['r2'], lgb_metrics['Test']['r2'], cat_metrics['Test']['r2']],
    'MAPE (%)': [xgb_metrics['Test']['mape'], lgb_metrics['Test']['mape'], cat_metrics['Test']['mape']]
})

results = results.sort_values('RMSE')
print(results.to_string(index=False))
print("\nBest model by RMSE: " + results.iloc[0]['Model'])
print(f"   RMSE: ${results.iloc[0]['RMSE']:,.2f} | R²: {results.iloc[0]['R²']:.4f} | MAPE: {results.iloc[0]['MAPE (%)']:.2f}%")


------------------------------
Basic models comparison
------------------------------
   Model         RMSE       R²  MAPE (%)
 XGBoost 14905.261199 0.917496  9.476430
CatBoost 15259.086137 0.913533  9.972207
LightGBM 15341.100478 0.912601  9.668906

🏆 Лучшая модель по RMSE: XGBoost
   RMSE: $14,905.26 | R²: 0.9175 | MAPE: 9.48%


In [100]:
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# ===============================
# OPTUNA HYPERPARAMETER TUNING - MULTI-METRIC
# ===============================

def objective_xgboost(trial, metric='rmse'):
    """XGBoost hyperparameter optimization"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 2500, step=250),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'random_state': 42,
        'tree_method': 'hist',
        'early_stopping_rounds': 50
    }

    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

    y_pred = model.predict(X_val)

    if metric == 'rmse':
        return np.sqrt(mean_squared_error(y_val, y_pred))
    else:  # mape
        y_val_original = np.expm1(y_val)
        y_pred_original = np.expm1(y_pred)
        return mean_absolute_percentage_error(y_val_original, y_pred_original)


def objective_lightgbm(trial, metric='rmse'):
    """LightGBM hyperparameter optimization"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 2500, step=250),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42,
        'verbose': -1
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=0)]
    )

    y_pred = model.predict(X_val)

    if metric == 'rmse':
        return np.sqrt(mean_squared_error(y_val, y_pred))
    else:  # mape
        y_val_original = np.expm1(y_val)
        y_pred_original = np.expm1(y_pred)
        return mean_absolute_percentage_error(y_val_original, y_pred_original)


def objective_catboost(trial, metric='rmse'):
    """CatBoost hyperparameter optimization"""
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2500, step=250),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'random_seed': 42,
        'verbose': False,
        'early_stopping_rounds': 50
    }

    cat_features_indices = [X_cat.columns.get_loc(col) for col in categorical_features]
    params['cat_features'] = cat_features_indices

    model = CatBoostRegressor(**params)
    model.fit(
        X_train_cat, y_train_cat,
        eval_set=(X_val_cat, y_val_cat),
        use_best_model=True
    )

    y_pred = model.predict(X_val_cat)

    if metric == 'rmse':
        return np.sqrt(mean_squared_error(y_val_cat, y_pred))
    else:  # mape
        y_val_original = np.expm1(y_val_cat)
        y_pred_original = np.expm1(y_pred)
        return mean_absolute_percentage_error(y_val_original, y_pred_original)


# ===============================
# 🔍 RUN OPTIMIZATION FOR BOTH METRICS
# ===============================

print("="*70)
print("🚀 STARTING DUAL-METRIC HYPERPARAMETER OPTIMIZATION WITH OPTUNA")
print("="*70)

OPTIMIZE_XGBOOST = True
OPTIMIZE_LIGHTGBM = True
OPTIMIZE_CATBOOST = True

N_TRIALS = 100

results_dict = {}

# ===============================
# XGBoost Optimization
# ===============================
if OPTIMIZE_XGBOOST:
    print("\n" + "="*70)
    print("⚡ Optimizing XGBoost for RMSE and MAPE...")
    print("="*70)

    # Optimize for RMSE
    print("\n🎯 Optimizing for RMSE...")
    study_xgb_rmse = optuna.create_study(direction='minimize', study_name='XGBoost_RMSE')
    study_xgb_rmse.optimize(lambda trial: objective_xgboost(trial, 'rmse'),
                            n_trials=N_TRIALS, show_progress_bar=True)

    # Optimize for MAPE
    print("\n🎯 Optimizing for MAPE...")
    study_xgb_mape = optuna.create_study(direction='minimize', study_name='XGBoost_MAPE')
    study_xgb_mape.optimize(lambda trial: objective_xgboost(trial, 'mape'),
                            n_trials=N_TRIALS, show_progress_bar=True)

    print(f"\n✅ XGBoost Best RMSE (validation): {study_xgb_rmse.best_value:.6f}")
    print(f"✅ XGBoost Best MAPE (validation): {study_xgb_mape.best_value:.4f}")

    # Train both models
    best_xgb_rmse = xgb.XGBRegressor(**study_xgb_rmse.best_params, random_state=42, tree_method='hist')
    best_xgb_rmse.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

    best_xgb_mape = xgb.XGBRegressor(**study_xgb_mape.best_params, random_state=42, tree_method='hist')
    best_xgb_mape.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

    # Evaluate on test set
    y_pred_xgb_rmse = np.expm1(best_xgb_rmse.predict(X_test))
    y_pred_xgb_mape = np.expm1(best_xgb_mape.predict(X_test))
    y_test_original = np.expm1(y_test)

    results_dict['XGBoost_RMSE'] = {
        'RMSE': np.sqrt(mean_squared_error(y_test_original, y_pred_xgb_rmse)),
        'R²': r2_score(y_test_original, y_pred_xgb_rmse),
        'MAPE': mean_absolute_percentage_error(y_test_original, y_pred_xgb_rmse) * 100,
        'model': best_xgb_rmse,
        'study': study_xgb_rmse,
        'params': study_xgb_rmse.best_params,
        'optimized_for': 'RMSE'
    }

    results_dict['XGBoost_MAPE'] = {
        'RMSE': np.sqrt(mean_squared_error(y_test_original, y_pred_xgb_mape)),
        'R²': r2_score(y_test_original, y_pred_xgb_mape),
        'MAPE': mean_absolute_percentage_error(y_test_original, y_pred_xgb_mape) * 100,
        'model': best_xgb_mape,
        'study': study_xgb_mape,
        'params': study_xgb_mape.best_params,
        'optimized_for': 'MAPE'
    }

# ===============================
# LightGBM Optimization
# ===============================
if OPTIMIZE_LIGHTGBM:
    print("\n" + "="*70)
    print("⚡ Optimizing LightGBM for RMSE and MAPE...")
    print("="*70)

    # Optimize for RMSE
    print("\n🎯 Optimizing for RMSE...")
    study_lgb_rmse = optuna.create_study(direction='minimize', study_name='LightGBM_RMSE')
    study_lgb_rmse.optimize(lambda trial: objective_lightgbm(trial, 'rmse'),
                            n_trials=N_TRIALS, show_progress_bar=True)

    # Optimize for MAPE
    print("\n🎯 Optimizing for MAPE...")
    study_lgb_mape = optuna.create_study(direction='minimize', study_name='LightGBM_MAPE')
    study_lgb_mape.optimize(lambda trial: objective_lightgbm(trial, 'mape'),
                            n_trials=N_TRIALS, show_progress_bar=True)

    print(f"\n✅ LightGBM Best RMSE (validation): {study_lgb_rmse.best_value:.6f}")
    print(f"✅ LightGBM Best MAPE (validation): {study_lgb_mape.best_value:.4f}")

    # Train both models
    best_lgb_rmse = lgb.LGBMRegressor(**study_lgb_rmse.best_params, random_state=42, verbose=-1)
    best_lgb_rmse.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                      callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=0)])

    best_lgb_mape = lgb.LGBMRegressor(**study_lgb_mape.best_params, random_state=42, verbose=-1)
    best_lgb_mape.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                      callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=0)])

    y_pred_lgb_rmse = np.expm1(best_lgb_rmse.predict(X_test))
    y_pred_lgb_mape = np.expm1(best_lgb_mape.predict(X_test))

    results_dict['LightGBM_RMSE'] = {
        'RMSE': np.sqrt(mean_squared_error(y_test_original, y_pred_lgb_rmse)),
        'R²': r2_score(y_test_original, y_pred_lgb_rmse),
        'MAPE': mean_absolute_percentage_error(y_test_original, y_pred_lgb_rmse) * 100,
        'model': best_lgb_rmse,
        'study': study_lgb_rmse,
        'params': study_lgb_rmse.best_params,
        'optimized_for': 'RMSE'
    }

    results_dict['LightGBM_MAPE'] = {
        'RMSE': np.sqrt(mean_squared_error(y_test_original, y_pred_lgb_mape)),
        'R²': r2_score(y_test_original, y_pred_lgb_mape),
        'MAPE': mean_absolute_percentage_error(y_test_original, y_pred_lgb_mape) * 100,
        'model': best_lgb_mape,
        'study': study_lgb_mape,
        'params': study_lgb_mape.best_params,
        'optimized_for': 'MAPE'
    }

# ===============================
# CatBoost Optimization
# ===============================
if OPTIMIZE_CATBOOST:
    print("\n" + "="*70)
    print("⚡ Optimizing CatBoost for RMSE and MAPE...")
    print("="*70)

    # Optimize for RMSE
    print("\n🎯 Optimizing for RMSE...")
    study_cat_rmse = optuna.create_study(direction='minimize', study_name='CatBoost_RMSE')
    study_cat_rmse.optimize(lambda trial: objective_catboost(trial, 'rmse'),
                            n_trials=N_TRIALS, show_progress_bar=True)

    # Optimize for MAPE
    print("\n🎯 Optimizing for MAPE...")
    study_cat_mape = optuna.create_study(direction='minimize', study_name='CatBoost_MAPE')
    study_cat_mape.optimize(lambda trial: objective_catboost(trial, 'mape'),
                            n_trials=N_TRIALS, show_progress_bar=True)

    print(f"\n✅ CatBoost Best RMSE (validation): {study_cat_rmse.best_value:.6f}")
    print(f"✅ CatBoost Best MAPE (validation): {study_cat_mape.best_value:.4f}")

    # Train both models
    cat_features_indices = [X_cat.columns.get_loc(col) for col in categorical_features]

    best_cat_rmse_params = study_cat_rmse.best_params.copy()
    best_cat_rmse_params['cat_features'] = cat_features_indices
    best_cat_rmse_params['verbose'] = False
    best_cat_rmse = CatBoostRegressor(**best_cat_rmse_params)
    best_cat_rmse.fit(X_train_cat, y_train_cat, eval_set=(X_val_cat, y_val_cat), use_best_model=True)

    best_cat_mape_params = study_cat_mape.best_params.copy()
    best_cat_mape_params['cat_features'] = cat_features_indices
    best_cat_mape_params['verbose'] = False
    best_cat_mape = CatBoostRegressor(**best_cat_mape_params)
    best_cat_mape.fit(X_train_cat, y_train_cat, eval_set=(X_val_cat, y_val_cat), use_best_model=True)

    y_pred_cat_rmse = np.expm1(best_cat_rmse.predict(X_test_cat))
    y_pred_cat_mape = np.expm1(best_cat_mape.predict(X_test_cat))
    y_test_cat_original = np.expm1(y_test_cat)

    results_dict['CatBoost_RMSE'] = {
        'RMSE': np.sqrt(mean_squared_error(y_test_cat_original, y_pred_cat_rmse)),
        'R²': r2_score(y_test_cat_original, y_pred_cat_rmse),
        'MAPE': mean_absolute_percentage_error(y_test_cat_original, y_pred_cat_rmse) * 100,
        'model': best_cat_rmse,
        'study': study_cat_rmse,
        'params': study_cat_rmse.best_params,
        'optimized_for': 'RMSE'
    }

    results_dict['CatBoost_MAPE'] = {
        'RMSE': np.sqrt(mean_squared_error(y_test_cat_original, y_pred_cat_mape)),
        'R²': r2_score(y_test_cat_original, y_pred_cat_mape),
        'MAPE': mean_absolute_percentage_error(y_test_cat_original, y_pred_cat_mape) * 100,
        'model': best_cat_mape,
        'study': study_cat_mape,
        'params': study_cat_mape.best_params,
        'optimized_for': 'MAPE'
    }

# ===============================
# 📊 COMPREHENSIVE COMPARISON
# ===============================
print("\n" + "="*70)
print("🏆 DUAL-METRIC OPTIMIZATION - FINAL TEST SET RESULTS")
print("="*70)

comparison_df = pd.DataFrame({
    'Model': list(results_dict.keys()),
    'Optimized_For': [results_dict[k]['optimized_for'] for k in results_dict.keys()],
    'RMSE': [results_dict[k]['RMSE'] for k in results_dict.keys()],
    'R²': [results_dict[k]['R²'] for k in results_dict.keys()],
    'MAPE (%)': [results_dict[k]['MAPE'] for k in results_dict.keys()]
})

print("\n📋 All Models Comparison:")
print(comparison_df.to_string(index=False))

# Find best for each metric
print("\n" + "="*70)
print("🥇 BEST MODELS BY METRIC")
print("="*70)

best_rmse_idx = comparison_df['RMSE'].idxmin()
best_mape_idx = comparison_df['MAPE (%)'].idxmin()

print(f"\n🎯 Best RMSE Model: {comparison_df.loc[best_rmse_idx, 'Model']}")
print(f"   RMSE: ${comparison_df.loc[best_rmse_idx, 'RMSE']:,.2f}")
print(f"   R²: {comparison_df.loc[best_rmse_idx, 'R²']:.4f}")
print(f"   MAPE: {comparison_df.loc[best_rmse_idx, 'MAPE (%)']:.2f}%")

print(f"\n🎯 Best MAPE Model: {comparison_df.loc[best_mape_idx, 'Model']}")
print(f"   RMSE: ${comparison_df.loc[best_mape_idx, 'RMSE']:,.2f}")
print(f"   R²: {comparison_df.loc[best_mape_idx, 'R²']:.4f}")
print(f"   MAPE: {comparison_df.loc[best_mape_idx, 'MAPE (%)']:.2f}%")

# ===============================
# 📋 PARAMETER COMPARISON
# ===============================
print("\n" + "="*70)
print("📋 PARAMETER COMPARISON FOR EACH MODEL")
print("="*70)

for model_type in ['XGBoost', 'LightGBM', 'CatBoost']:
    rmse_key = f"{model_type}_RMSE"
    mape_key = f"{model_type}_MAPE"

    if rmse_key in results_dict and mape_key in results_dict:
        print(f"\n{model_type} Parameter Comparison:")
        print("-" * 70)

        rmse_params = results_dict[rmse_key]['params']
        mape_params = results_dict[mape_key]['params']

        all_params = set(list(rmse_params.keys()) + list(mape_params.keys()))

        for param in sorted(all_params):
            rmse_val = rmse_params.get(param, 'N/A')
            mape_val = mape_params.get(param, 'N/A')
            diff = '✓ Same' if rmse_val == mape_val else '✗ Different'
            print(f"  {param:20s} | RMSE: {str(rmse_val):15s} | MAPE: {str(mape_val):15s} | {diff}")

print("\n✅ Dual-Metric Optimization Complete!")
print("\n💡 Key Insights:")
print("   - Each model has been optimized separately for RMSE and MAPE")
print("   - Compare parameters to understand trade-offs between metrics")
print("   - Choose the model based on your business objective")
print("   - Models are stored in results_dict[model_name]['model']")

[I 2026-01-18 23:22:12,301] A new study created in memory with name: XGBoost_RMSE


🚀 STARTING DUAL-METRIC HYPERPARAMETER OPTIMIZATION WITH OPTUNA

⚡ Optimizing XGBoost for RMSE and MAPE...

🎯 Optimizing for RMSE...


Best trial: 0. Best value: 0.130338:   1%|          | 1/100 [00:03<06:10,  3.74s/it]

[I 2026-01-18 23:22:16,042] Trial 0 finished with value: 0.13033765094227379 and parameters: {'n_estimators': 1750, 'learning_rate': 0.026049707771340205, 'max_depth': 6, 'min_child_weight': 4, 'subsample': 0.8887256573982785, 'colsample_bytree': 0.6854609190627347, 'reg_alpha': 1.1606736246138418e-06, 'reg_lambda': 4.279147574229222e-06, 'gamma': 9.406842207440751e-07}. Best is trial 0 with value: 0.13033765094227379.


Best trial: 0. Best value: 0.130338:   2%|▏         | 2/100 [00:08<06:37,  4.05s/it]

...

[I 2026-01-19 00:42:32,080] Trial 97 finished with value: 0.09717889499263738 and parameters: {'iterations': 2500, 'learning_rate': 0.07820261151376594, 'depth': 9, 'l2_leaf_reg': 3.278310486092924e-08, 'subsample': 0.8472532852167485}. Best is trial 29 with value: 0.09465064882990945.


Best trial: 29. Best value: 0.0946506:  99%|█████████▉| 99/100 [12:46<00:10, 10.33s/it]

[I 2026-01-19 00:42:43,943] Trial 98 finished with value: 0.09688975346849857 and parameters: {'iterations': 2000, 'learning_rate': 0.024596040409296582, 'depth': 8, 'l2_leaf_reg': 6.47319845584135e-08, 'subsample': 0.8282858805953508}. Best is trial 29 with value: 0.09465064882990945.


Best trial: 29. Best value: 0.0946506: 100%|██████████| 100/100 [12:56<00:00,  7.77s/it]


[I 2026-01-19 00:42:54,201] Trial 99 finished with value: 0.09597598394377328 and parameters: {'iterations': 2000, 'learning_rate': 0.043791951955836225, 'depth': 8, 'l2_leaf_reg': 2.8653956394725513e-07, 'subsample': 0.8599659478578129}. Best is trial 29 with value: 0.09465064882990945.

✅ CatBoost Best RMSE (validation): 0.127916
✅ CatBoost Best MAPE (validation): 0.0947

🏆 DUAL-METRIC OPTIMIZATION - FINAL TEST SET RESULTS

📋 All Models Comparison:
        Model Optimized_For         RMSE       R²  MAPE (%)
 XGBoost_RMSE          RMSE 14965.941129 0.916823  9.336905
 XGBoost_MAPE          MAPE 14757.846477 0.919120  9.245500
LightGBM_RMSE          RMSE 15124.214839 0.915055  9.593321
LightGBM_MAPE          MAPE 15036.856976 0.916033  9.225793
CatBoost_RMSE          RMSE 14817.238706 0.918468  9.525816
CatBoost_MAPE          MAPE 14815.892530 0.918483  9.554846

🥇 BEST MODELS BY METRIC

🎯 Best RMSE Model: XGBoost_MAPE
   RMSE: $14,757.85
   R²: 0.9191
   MAPE: 9.25%

🎯 Best MAPE Model

Best RMSE Model: XGBoost_MAPE

In [102]:
optimal_params = {
    "colsample_bytree": 0.6174243195963651,
    "gamma": 0.00029194695300892537,
    "learning_rate": 0.025072744653519028,
    "max_depth": 8,
    "min_child_weight": 3,
    "n_estimators": 2000,
    "reg_alpha": 0.017313707755127073,
    "reg_lambda": 0.07549915003331413,
    "subsample": 0.8978305041846237,
    "random_state": 42,
    "tree_method": 'hist',
    "early_stopping_rounds": 50
}

print("\nOptimal Hyperparameters:")
for key, value in optimal_params.items():
    if key != 'early_stopping_rounds':
        print(f"  {key:20s}: {value}")

model = xgb.XGBRegressor(**optimal_params)

# Train with validation monitoring
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=50
)


Optimal Hyperparameters:
  colsample_bytree    : 0.6174243195963651
  gamma               : 0.00029194695300892537
  learning_rate       : 0.025072744653519028
  max_depth           : 8
  min_child_weight    : 3
  n_estimators        : 2000
  reg_alpha           : 0.017313707755127073
  reg_lambda          : 0.07549915003331413
  subsample           : 0.8978305041846237
  random_state        : 42
  tree_method         : hist
[0]	validation_0-rmse:0.52081
[50]	validation_0-rmse:0.22550
[100]	validation_0-rmse:0.16227
[150]	validation_0-rmse:0.14734
[200]	validation_0-rmse:0.14169
[250]	validation_0-rmse:0.13844
[300]	validation_0-rmse:0.13643
[350]	validation_0-rmse:0.13541
[400]	validation_0-rmse:0.13456
[450]	validation_0-rmse:0.13385
[500]	validation_0-rmse:0.13321
[550]	validation_0-rmse:0.13283
[600]	validation_0-rmse:0.13249
[650]	validation_0-rmse:0.13222
[700]	validation_0-rmse:0.13201
[750]	validation_0-rmse:0.13173
[800]	validation_0-rmse:0.13160
[850]	validation_0-rmse:0.131

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.6174243195963651
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",50
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import l

In [103]:
print(f"\nTraining complete! Best iteration: {model.best_iteration}")


Training complete! Best iteration: 1600


In [106]:
metrics = evaluate_model(model, 'XGBoost (MAPE Optimised)', X_train, y_train, X_val, y_val, X_test,y_test)


Results for: XGBoost (MAPE Optimised)

Train Set:
  RMSE: $2,616.40
  MAE:  $1,564.75
  R²:   0.9974
  MAPE: 1.80%

Mean Error by Price Bin
------------------------------
         mean_abs_error  mean_abs_pct_error
bin                                        
<50k         630.799452            1.610169
50–80k      1143.472686            1.789704
80–110k     1748.937346            1.876919
>110k       3183.867401            1.943287

Validation Set:
  RMSE: $15,555.23
  MAE:  $8,601.91
  R²:   0.9097
  MAPE: 9.45%

Mean Error by Price Bin
------------------------------
         mean_abs_error  mean_abs_pct_error
bin                                        
<50k        3314.592287            9.101499
50–80k      5535.087787            8.659122
80–110k     9026.873224            9.647259
>110k      19164.748051           11.015151

Test Set:
  RMSE: $14,781.55
  MAE:  $8,354.80
  R²:   0.9189
  MAPE: 9.27%

Mean Error by Price Bin
------------------------------
         mean_abs_error  mea

In [107]:
import pickle

print("\n" + "=" * 70)
print("SAVING MODEL AND ARTIFACTS")
print("=" * 70)

# Save complete model package
model_package = {
    'model': model,
    'encoder': ohe,
    'feature_names': {
        'numeric': numeric_features,
        'categorical': categorical_features
    },
    'hyperparameters': optimal_params,
    'metrics': {
        'train': metrics['Train'],
        'validation': metrics['Validation'],
        'test': metrics['Test']
    },
    'metadata': {
        'model_type': 'XGBoost',
        'optimization_metric': 'RMSE',
        'training_samples': len(X_train),
        'features_count': X.shape[1]
    }
}

with open('astana_price_model.pkl', 'wb') as f:
    pickle.dump(model_package, f)

print("\nModel saved to: astana_price_model.pkl")
print("\nPackage contents:")
print("  - Trained XGBoost model")
print("  - OneHotEncoder for categorical features")
print("  - Feature names and types")
print("  - Optimal hyperparameters")
print("  - Performance metrics")
print("  - Training metadata")


SAVING MODEL AND ARTIFACTS

Model saved to: astana_price_model.pkl

Package contents:
  - Trained XGBoost model
  - OneHotEncoder for categorical features
  - Feature names and types
  - Optimal hyperparameters
  - Performance metrics
  - Training metadata
